In [96]:
import pandas as pd
import numpy as np
import random
import toolsets.data_prep as data_prep
%load_ext autoreload
%autoreload 2
descriptors = pd.read_csv("data/multiRT/MultiRT_hilic_descriptors.csv", low_memory=False)
descriptors = descriptors.select_dtypes(exclude=['object'])

i am updated!
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
combined_data = pd.read_csv("data/multiRT/Combined dataset.csv")
hilic = combined_data.loc[combined_data['Column']=='HILIC']
hilic.reset_index(inplace=True, drop=True)
# hilic=data_prep.dataset_prep(hilic)
# hilic = data_prep.make_split_index(hilic)

In [103]:
data = pd.concat([hilic, descriptors],axis =1)

In [104]:
data = data_prep.dataset_prep(data)

In [105]:
train, test = data_prep.make_train_test(data, 0.8,0.2)

In [116]:
num_features = train.select_dtypes(include = ["float64", "int64"]).columns

In [138]:
non_cat_features = train.select_dtypes(include = [ "bool",'object']).columns
# x[non_cat_features]=x[non_cat_features].astype('category')

i am updated!


In [148]:
num_features = train.select_dtypes(include = ["float64"]).columns

In [154]:
x, y = data_prep.make_x_y(train)

i am updated!


In [155]:
x

,pH,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,Organic_modifier_Methanol,Buffer_Formic acid,Lipinski_True,GhoseFilter_True
0,2.7,-0.034908,-0.4068,0,0,0.020457,1.272155,1.455625,0.020457,1.894151,...,428,33,0.117533,0.275060,-1.054626,-0.361782,0,1,1,1
1,3.0,-0.034908,-0.4068,0,0,0.020457,1.272155,1.455625,0.020457,1.894151,...,428,33,0.117533,0.275060,-1.054626,-0.361782,0,0,1,1
2,3.0,-0.034908,-0.4068,0,0,0.020457,1.272155,1.455625,0.020457,1.894151,...,428,33,0.117533,0.275060,-1.054626,-0.361782,0,1,1,1
3,5.0,-0.034908,-0.4068,0,0,0.020457,1.272155,1.455625,0.020457,1.894151,...,428,33,0.117533,0.275060,-1.054626,-0.361782,0,0,1,1
4,5.0,-0.034908,-0.4068,0,0,0.020457,1.272155,1.455625,0.020457,1.894151,...,428,33,0.117533,0.275060,-1.054626,-0.361782,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,5.0,1.758561,1.9274,3,0,1.873284,0.144229,0.221831,1.873284,0.376195,...,3122,49,1.577985,1.463666,1.547606,2.068563,0,1,1,1
832,2.7,1.758561,1.9274,3,0,1.873284,0.144229,0.221831,1.873284,0.376195,...,3122,49,1.577985,1.463666,1.547606,2.068563,1,1,1,1
833,3.0,1.758561,1.9274,3,0,1.873284,0.144229,0.221831,1.873284,0.376195,...,3122,49,1.577985,1.463666,1.547606,2.068563,1,1,1,1
834,5.0,1.758561,1.9274,3,0,1.873284,0.144229,0.221831,1.873284,0.376195,...,3122,49,1.577985,1.463666,1.547606,2.068563,1,0,1,1


# start of reproducing clean lab code

In [16]:
SEED = 123456
np.random.seed(SEED)
random.seed(SEED)

In [17]:
from sklearn.datasets import fetch_openml

data = fetch_openml("credit-g")  # get the credit data from OpenML
X_raw = data.data  # features (pandas DataFrame)
y_raw = data.target  # labels (pandas Series)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [135]:
(y_raw)

0      good
1       bad
2      good
3      good
4       bad
       ... 
995    good
996    good
997    good
998     bad
999    good
Name: class, Length: 1000, dtype: category
Categories (2, object): ['good', 'bad']

In [72]:
from sklearn.preprocessing import StandardScaler

In [79]:
cat_features = X_raw.select_dtypes("category").columns
X_encoded = pd.get_dummies(X_raw, columns=cat_features, drop_first=True)

In [81]:
num_features = X_raw.select_dtypes("float64").columns
scaler = StandardScaler()
X_scaled = X_encoded.copy()
X_scaled[num_features] = scaler.fit_transform(X_encoded[num_features])

y = y_raw.map({"bad": 0, "good": 1})  # encode labels as integers

In [130]:
y

0      1
1      0
2      1
3      1
4      0
      ..
995    1
996    1
997    1
998    0
999    1
Name: class, Length: 1000, dtype: category
Categories (2, int64): [1, 0]

In [86]:
X_raw['duration']

0       6.0
1      48.0
2      12.0
3      42.0
4      24.0
       ... 
995    12.0
996    30.0
997    12.0
998    45.0
999    45.0
Name: duration, Length: 1000, dtype: float64

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
from sklearn.model_selection import cross_val_predict

num_crossval_folds = 3  # for efficiency; values like 5 or 10 will generally work better
pred_probs = cross_val_predict(
    clf,
    X_scaled,
    y,
    cv=num_crossval_folds,
    method="predict_proba",
)